In [72]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
import xml.etree.ElementTree as et
from io import BytesIO

# Женское население

In [89]:
#Pull the filters out of the html code
def get_filters_raw(id):
    """
    Код принимает на вход id показателя и возвращает список с фильтрами
    Это вспомогательная функция
    """
    try: 
        html = requests.get(f'https://www.fedstat.ru/indicator/{id}')
        soup = BeautifulSoup(html.text, "lxml")
        script = soup.find_all("script")[11].text
        pattern = r"filters:\s*(\{.*?\})(?=\s*,\s*left_columns)"
        match = re.search(pattern, script, re.DOTALL).group(0)
        filters = "{" + match + "}"
        filters = re.sub(r'([{,]\s*)(\w+)(\s*:)', r'\1"\2"\3', filters)
        filters = filters.replace("'", '"')
        filters = json.loads(filters)['filters']
        return filters
    except requests.RequestException as e:
        raise requests.RequestException(f"Ошибка HTTP-запроса: {str(e)}")

def get_filters_text(filters):
    """
    Возвращает код и название показателя, доступного для фильтрации
    """
    filter_values = {key : filters[key]['title'] for key in list(filters.keys())[1:]}
    return filter_values

def get_values(filters):
    """
    Возвращает названия и коды доступных значений для выбранных фильтров
    """
    filter_codes = filters.keys()
    categories =[]
    for key in filter_codes:
        categories.append({
            key: list(filters[key]['values'].keys())
        })
    ids = [f"{k}_{val}" for item in categories for k, v in item.items() for val in v]
    return ids

def load_indicator(id, data_type, ids):
    """
    На вход передается код индикатора, тип данных, коды показателей для фильтрации
    Возвращает датафрейм с данными
    """
    data = {
            "lineObjectIds": ["0",  "57831", "58335", "58274", "30611"],
            "columnObjectIds": ["3", "33560"],
            "selectedFilterIds": [
                ids
            ]
        }
    params = {
        "format" : data_type,
        "id" : id
    }
    try: 
        response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
        if "excel" in response.get("headers"):
            return pd.read_excel(BytesIO(response.content), engine = "xlrd")
        else:
            raise ValueError(
                f"Ответ не содержит {data_type} файла"
            )
    except requests.RequestException as e:
        raise requests.RequestException(f"Ошибка HTTP-запроса: {str(e)}")  

In [ ]:

def get_min_age(age):
    numbers = re.findall(r'\d+', age)
    return int(numbers[0]) if numbers else None
    
def get_max_age(age):
    numbers = re.findall(r"\d+", age)
    return int(numbers[-1]) if numbers else None
    
def categorize_age(age_str):
    age_str = age_str.lower()
    numbers = re.findall(r'\d+', age_str)
    words = re.findall(r'\b[а-яА-ЯёЁ]+\b', age_str, flags=re.IGNORECASE)
    if "-" in age_str and len(words) == 1:
        age_diff = int(numbers[1]) - int(numbers[0])
        return 2 if age_diff == 4 else 3
    elif "всего" in age_str or not numbers or len(words) > 3:
        return 4
    else:
        return 1

def process_file(df):
    #Remove unnecassary columns
    df.drop([df.columns[0], df.columns[4]], axis = 1, inplace = True)
    df = df.drop(0, axis = 0).reset_index(drop = True)
    df.rename(columns = {
        'Unnamed: 1' : "region",
        "Unnamed: 2" : "age",
        "Unnamed: 3" : "settlement"
    }, inplace = True)

    df.region = df.region.str.strip()

    rm_districts = '|'.join([
    r'Южный федеральный',
    r'Сибирский федеральный',
    r'Северо-Кавказский федеральный',
    r'Дальневосточный федеральный'
    ])
    districts_mask = df['region'].str.contains(rm_districts, case=False)
    df= df[~districts_mask].reset_index(drop = True)
    
    years = [col for col in df.columns if col.isdigit()]
    df[years] = df[years].astype("Int64")
    
    df['min_age']  = df['age'].apply(get_min_age)
    df['max_age'] = df['age'].apply(get_max_age)
    df['age_category'] = df['age'].apply(categorize_age)

In [ ]:
filters = get_filters(id = 33459)
filter_values = get_filters_text(filters)
filter_ids = get_values(filters)
response = load_indicator(id = 33459, data_type = "excel", ids = filter_ids)

<Response [200]>

In [86]:
df = load_to_excel(response.content)

## SDMX data

In [ ]:
#All values
data = {
        "lineObjectIds": ["0", "30611", "58335", "57831", "58274"],
        "columnObjectIds": ["3", "33560"],
        "selectedFilterIds": ids
}
params = {
    "format" : "sdmx",
    "id" : "33459"
}
response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
print(response.status_code, response.headers)

In [7]:
# Adding  regions
regions = [f"{k}_{val}" for item in categories for k, v in item.items() if k =="57831" for val in v]

In [8]:
#Adding years
years = [f"{k}_{val}" for item in categories for k, v in item.items() if k =="3" for val in v]

In [9]:
#Adding settlement type
settlement_type = [f"{k}_{val}" for item in categories for k, v in item.items() if k =="58274" for val in v]
settlement_type

['58274_1744150', '58274_1750789', '58274_1750788']

In [10]:
#Adding age categories
age =  [f"{k}_{val}" for item in categories for k, v in item.items() if k =="58335" for val in v]

In [12]:
#collect regional data
data = {
        "lineObjectIds": ["0", "30611", "58335", "57831", "58274"],
        "columnObjectIds": ["3", "33560"],
        "selectedFilterIds": [
            "0_33459",
            years,
            "30611_950458",
            "33560_1540248",
            regions,
            "58274_1744150",
            "58335_1709566"
        ]
    }
params = {
    "format" : "sdmx",
    "id" : "33459"
}
response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
print(response.status_code, response.headers)

200 {'Content-Type': 'text/xml', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Sat, 12 Jul 2025 10:39:48 GMT', 'Set-Cookie': 'JSESSIONID=20E85FCD89FE3297FE03E7FAF9608248; Path=/; Secure; HttpOnly, session-cookie=18517ad4dc4c6a7edcd5a505204218c4d2506ea5bd73d1c7df057e3ecf38e193bc79ec9cdd66ebf6efc4768835a5ee4b; Max-Age=86400; Path=/; secure; HttpOnly', 'Cache-Control': 'no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': '0', 'Pragma': 'public', 'Content-Disposition': "attachment; filename*=UTF-8''data.xml", 'vary': 'accept-encoding', 'Content-Language': 'ru', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'Content-Security-Policy': 'upgrade-insecure-requests', 'Referrer-Policy': 'strict-origin'}


In [13]:
# Save the data in a file
with open("regions.xml", "wb") as file:
    file.write(response.content)

In [41]:
#Transform sdmx file into pandas dataframe
def read_sdmx(filename):
    tree = et.parse(filename)
    root = tree.getroot()

    namespaces = {
        "generic": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/generic",
        "message": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/message",
        "structure": 'http://www.SDMX.org/resources/SDMXML/schemas/v1_0/structure'
    } 

    dataset = root.find(".//message:DataSet", namespaces)
    data = []
    for series in dataset.findall(".//generic:Series", namespaces):
        observations = series.findall(".//generic:Obs", namespaces)
        pair = {}
        for obs in observations:
            time = obs.find(".//generic:Time", namespaces).text
            value = obs.find(".//generic:ObsValue", namespaces).attrib['value']
            region = obs.find(".//")
            data.append({
                "year" : time,
                "population" : value
                })
    return pd.DataFrame(data)

data = read_sdmx("regions.xml")
data

,year,population
0,1990,78549601
1,1991,78817695
2,1992,78914788
3,1993,78899303
4,1994,78770458
...,...,...
3502,1996,3833075
3503,1999,3700204
3504,2000,3620686
3505,2001,3620686


## Excel data

In [40]:
#Let's get the data in excel
def load_indicator(id, data_type, regions, years, settlement_type, age):
    data = {
            "lineObjectIds": ["0",  "57831", "58335", "58274", "30611"],
            "columnObjectIds": ["3", "33560"],
            "selectedFilterIds": [
                regions,
                "0_33459",
                years,
                "30611_950458",
                "33560_1540248",
                settlement_type,
                age
            ]
        }
    params = {
        "format" : data_type,
        "id" : id
    }
    response = requests.post("https://www.fedstat.ru/indicator/data.do?", params = params, data = data)
    return response
# print(response.status_code, response.headers)

In [ ]:
#Save the excel data
with open("all_data.xls", "wb") as file:
    file.write(response.content)

In [17]:
#Load the data
df = pd.read_excel("all_data.xls", header = 2)
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,1990,1991,1992,1993,1994,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,NaN,NaN,NaN,NaN,NaN,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,...,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января,на 1 января
1,Численность постоянного населения - женщин по ...,Российская Федерация,Всего,все население,человек,78549601,78817695,78914788,78899303,78770458,...,78958558,79137859,79207619,79202094,79219390,78952485,78601633,78354712,78233436,78154709
2,Численность постоянного населения - женщин по ...,Российская Федерация,Всего,городское население,человек,57983496,58303864,58267844,57878461,57688561,...,59279662,59512332,59660317,59755068,59858643,59691444,59487917,59352665,59312967,59399259
3,Численность постоянного населения - женщин по ...,Российская Федерация,Всего,сельское население,человек,20566105,20513831,20646944,21020842,21081897,...,19678896,19625527,19547302,19447026,19360747,19261041,19113716,19002047,18920469,18755450
4,Численность постоянного населения - женщин по ...,Российская Федерация,0,все население,человек,1035959,957048,868672,770771,675810,...,942238,910777,814676,771071,714593,688397,673494,632575,611254,593327


In [18]:
#Remove unnecassary columns
df.drop([df.columns[0], df.columns[4]], axis = 1, inplace = True)
df = df.drop(0, axis = 0).reset_index(drop = True)

In [19]:
# Rename
df.rename(columns = {
    'Unnamed: 1' : "region",
    "Unnamed: 2" : "age",
    "Unnamed: 3" : "settlement"
    }, inplace = True)

In [20]:
df.region = df.region.str.strip()

In [21]:
#Remove unnecessary districts
rm_districts = '|'.join([
    r'Южный федеральный',
    r'Сибирский федеральный',
    r'Северо-Кавказский федеральный',
    r'Дальневосточный федеральный'
])
districts_mask = df['region'].str.contains(rm_districts, case=False)
df= df[~districts_mask].reset_index(drop = True)

In [22]:
years = [col for col in df.columns if col.isdigit()]
df[years] = df[years].astype("Int64")

In [23]:
def get_min_age(age):
    numbers = re.findall(r'\d+', age)
    return int(numbers[0]) if numbers else None
    
def get_max_age(age):
    numbers = re.findall(r"\d+", age)
    return int(numbers[-1]) if numbers else None
    
def categorize_age(age_str):
    age_str = age_str.lower()
    numbers = re.findall(r'\d+', age_str)
    words = re.findall(r'\b[а-яА-ЯёЁ]+\b', age_str, flags=re.IGNORECASE)
    if "-" in age_str and len(words) == 1:
        age_diff = int(numbers[1]) - int(numbers[0])
        return 2 if age_diff == 4 else 3
    elif "всего" in age_str or not numbers or len(words) > 3:
        return 4
    else:
        return 1

df['min_age']  = df['age'].apply(get_min_age)
df['max_age'] = df['age'].apply(get_max_age)
df['age_category'] = df['age'].apply(categorize_age)

In [24]:
#Add mid-year value
min_year = min([int(col) for col in df.columns if col.isdigit()])
df_mid = pd.DataFrame()

for col in df.columns:
    if col.isdigit():
        if int(col) > min_year:
            df_mid[f"{col}mid"] = (df[f"{col}"] + df[f"{int(col)-1}"])/2
        df_mid[f"{col}end"] = df[col]
    else:
        df_mid[col] = df[col]
df_mid.head()

,region,age,settlement,1990end,1991mid,1991end,1992mid,1992end,1993mid,1993end,...,2022end,2023mid,2023end,2024mid,2024end,2025mid,2025end,min_age,max_age,age_category
0,Российская Федерация,Всего,все население,78549601,78683648.0,78817695,78866241.5,78914788,78907045.5,78899303,...,78601633,78478172.5,78354712,78294074.0,78233436,78194072.5,78154709,NaN,NaN,4
1,Российская Федерация,Всего,городское население,57983496,58143680.0,58303864,58285854.0,58267844,58073152.5,57878461,...,59487917,59420291.0,59352665,59332816.0,59312967,59356113.0,59399259,NaN,NaN,4
2,Российская Федерация,Всего,сельское население,20566105,20539968.0,20513831,20580387.5,20646944,20833893.0,21020842,...,19113716,19057881.5,19002047,18961258.0,18920469,18837959.5,18755450,NaN,NaN,4
3,Российская Федерация,0,все население,1035959,996503.5,957048,912860.0,868672,819721.5,770771,...,673494,653034.5,632575,621914.5,611254,602290.5,593327,0.0,0.0,1
4,Российская Федерация,0,городское население,731046,698696.5,666347,630439.5,594532,555098.0,515664,...,506226,490632.5,475039,466675.0,458311,453286.0,448261,0.0,0.0,1


In [25]:
#Fix federal districts

dfo_regions = [
    'Республика Бурятия',
    'Забайкальский край',
    'Республика Саха (Якутия)',
    'Камчатский край',
    'Приморский край',
    'Хабаровский край',
    'Амурская область',
    'Магаданская область',
    'Сахалинская область',
    'Еврейская автономная область',
    'Чукотский автономный округ'
]
dfo_mask = df_mid['region'].isin(dfo_regions)
dfo = df_mid[dfo_mask]
dfo_agg = dfo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
dfo_agg.insert(0, 'region', 'Дальневосточный федеральный округ')

In [26]:
sfo_regions = ['Республика Алтай',
    'Республика Тыва',
    'Республика Хакасия',
    'Алтайский край',
    'Красноярский край',
    'Иркутская область',
    'Кемеровская область - Кузбасс',
    'Новосибирская область',
    'Омская область',
    'Томская область']
sfo_mask = df_mid['region'].isin(sfo_regions)
sfo = df_mid[sfo_mask]
sfo_agg = sfo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
sfo_agg.insert(0, 'region', 'Сибирский федеральный округ')

In [27]:
ufo_pattern = '|'.join([
    r'Адыгея', 
    r'Калмыкия',
    r'Краснодарский',
    r'Астраханская',
    r'Волгоградская',
    r'Ростовская',
    r'Крым',
    r'Севастополь'
])

skfo_pattern = '|'.join([
    r'Дагестан',
    r'Ингушетия',
    r'Кабардин',
    r'Балкар',
    r'Карачаев',
    r'Черкес',
    r'Осетия',
    r'Алания',
    r'Чечен',
    r'Ставрополь'
])

ufo_mask = df_mid['region'].str.contains(ufo_pattern, case = False, na = False)
ufo = df_mid[ufo_mask]
ufo_agg = ufo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
ufo_agg.insert(0, 'region', 'Южный федеральный округ')

skfo_mask = df_mid['region'].str.contains(skfo_pattern, case = False, na = False)
skfo = df_mid[skfo_mask]
skfo_agg = skfo.groupby(['age', 'settlement'], as_index= False).sum(numeric_only= True)
skfo_agg.insert(0, 'region', 'Северо-Кавказский федеральный округ')

df_mid = pd.concat([df_mid, ufo_agg, skfo_agg, sfo_agg], ignore_index = True)


In [28]:
df_mid = df_mid.groupby('region', sort = False).apply(
    lambda x: x.sort_values(['age_category', 'min_age', 'max_age'])
).reset_index(drop = True)

C:\Users\Denis\AppData\Local\Temp\ipykernel_9020\2471046149.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_mid = df_mid.groupby('region', sort = False).apply(


In [ ]:
#Highlight "total" rows with red color
def highlight_total(row, color = 'red'):
    return [f"background-color: {color}" for _ in row]
df_styled = df_mid.style.apply(
    lambda row : highlight_total(row) if row['age'] == 'Всего' else ['']*len(row),
    axis = 1
)
df_styled

In [ ]:
#Save in a csv file
df_mid.drop(columns = ['age_category', 'min_age', 'max_age'], inplace = True)
df_mid.to_excel("sorted_data.xlsx", index = False)
df_styled.to_excel("sorted_data_highlighted.xlsx", index = False)

# Мужское население

In [ ]:
get_filters(id = 31548)

{'Год': '3',
 'Возраст': '58335',
 'Классификатор объектов административно-территориального деления (ОКАТО)': '57831',
 'Единица измерения': '30611',
 'Период': '33560',
 'Тип поселения': '58274'}